In [7]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 25: Four-Dimensional Adventure ---</h2><p>The reindeer's symptoms are getting worse, and neither you nor the white-bearded man have a solution. At least the reindeer has a warm place to rest: a small bed near where you're sitting.</p>
<p>As you reach down, the reindeer looks up at you, <span title="It was not an accident.">accidentally</span> bumping a button on your wrist-mounted device with its nose in the process - a button labeled <em>"help"</em>.</p>
<p>"Hello, and welcome to the Time Travel Support Hotline! If you are lost in time and space, press 1. If you are trapped in a time paradox, press 2. If you need help caring for a sick reindeer, press 3. If you--"</p>
<p><em>Beep.</em></p>
<p>A few seconds later, you hear a new voice. "Hello; please state the nature of your reindeer." You try to describe the situation.</p>
<p>"Just a moment, I think I can remotely run a diagnostic scan." A beam of light projects from the device and sweeps over the reindeer a few times.</p>
<p>"Okay, it looks like your reindeer is very low on magical energy; it should fully recover if we can fix that.  Let me check your timeline for a source.... Got one. There's actually a powerful source of magical energy about 1000 years forward from you, and at roughly your position, too!  It looks like... hot chocolate?  Anyway, you should be able to travel there to pick some up; just don't forget a mug!  Is there anything else I can help you with today?"</p>
<p>You explain that your device isn't capable of going forward in time.  "I... see. That's tricky. Well, according to this information, your device should have the necessary hardware to open a small portal and send some hot chocolate back to you. You'll need a list of <em>fixed points in spacetime</em>; I'm transmitting it to you now."</p>
<p>"You just need to align your device to the constellations of fixed points so that it can lock on to the destination and open the portal. Let me look up how much hot chocolate that breed of reindeer needs."</p>
<p>"It says here that your particular reindeer is-- this can't be right, it says there's only one like that in the universe!  But THAT means that you're--" You disconnect the call.</p>
<p>The list of fixed points in spacetime (your puzzle input) is a set of four-dimensional coordinates. To align your device, acquire the hot chocolate, and save the reindeer, you just need to find the <em>number of constellations</em> of points in the list.</p>
<p>Two points are in the same <em>constellation</em> if their manhattan distance apart is <em>no more than 3</em> or if they can form a chain of points, each a manhattan distance no more than 3 from the last, between the two of them. (That is, if a point is close enough to a constellation, it "joins" that constellation.) For example:</p>
<pre><code> 0,0,0,0
 3,0,0,0
 0,3,0,0
 0,0,3,0
 0,0,0,3
 0,0,0,6
 9,0,0,0
12,0,0,0
</code></pre>
<p>In the above list, the first six points form a single constellation: <code>0,0,0,0</code> is exactly distance <code>3</code> from the next four, and the point at <code>0,0,0,6</code> is connected to the others by being <code>3</code> away from <code>0,0,0,3</code>, which is already in the constellation. The bottom two points, <code>9,0,0,0</code> and <code>12,0,0,0</code> are in a separate constellation because no point is close enough to connect them to the first constellation.  So, in the above list, the number of constellations is <code><em>2</em></code>.  (If a point at <code>6,0,0,0</code> were present, it would connect <code>3,0,0,0</code> and <code>9,0,0,0</code>, merging all of the points into a single giant constellation instead.)</p>
<p>In this example, the number of constellations is <code>4</code>:</p>
<pre><code>-1,2,2,0
0,0,2,-2
0,0,0,-2
-1,2,0,0
-2,-2,-2,2
3,0,2,-1
-1,3,2,2
-1,0,-1,0
0,2,1,-2
3,0,0,0
</code></pre>
<p>In this one, it's <code>3</code>:</p>
<pre><code>1,-1,0,1
2,0,-1,0
3,2,-1,0
0,0,3,1
0,0,-1,-1
2,3,-2,0
-2,2,0,0
2,-2,0,-1
1,-1,0,-1
3,2,0,2
</code></pre>
<p>Finally, in this one, it's <code>8</code>:</p>
<pre><code>1,-1,-1,-2
-2,-2,0,1
0,2,1,3
-2,3,-2,1
0,2,3,-2
-1,-1,1,-2
0,-2,-1,0
-2,2,3,-1
1,2,2,0
-1,-2,0,-2
</code></pre>
<p>The portly man nervously strokes his white beard. It's time to get that hot chocolate.</p>
<p><em>How many constellations are formed by the fixed points in spacetime?</em></p>
</article>


In [12]:
from collections import deque
from pprint import pprint
from typing import Sequence


tests = [
    {
        "name": "Example 1",
        "points": "\n".join(
            ",".join(map(str, l))
            for l in [
                [0, 0, 0, 0],
                [3, 0, 0, 0],
                [0, 3, 0, 0],
                [0, 0, 3, 0],
                [0, 0, 0, 3],
                [0, 0, 0, 6],
                [9, 0, 0, 0],
                [12, 0, 0, 0],
            ]
        ),
        "expected": 2,
    },
    {
        "name": "Example 2",
        "points": "\n".join(
            ",".join(map(str, l))
            for l in [
                [-1, 2, 2, 0],
                [0, 0, 2, -2],
                [0, 0, 0, -2],
                [-1, 2, 0, 0],
                [-2, -2, -2, 2],
                [3, 0, 2, -1],
                [-1, 3, 2, 2],
                [-1, 0, -1, 0],
                [0, 2, 1, -2],
                [3, 0, 0, 0],
            ]
        ),
        "expected": 4,
    },
    {
        "name": "Example 3",
        "points": "\n".join(
            ",".join(map(str, l))
            for l in [
                [1, -1, 0, 1],
                [2, 0, -1, 0],
                [3, 2, -1, 0],
                [0, 0, 3, 1],
                [0, 0, -1, -1],
                [2, 3, -2, 0],
                [-2, 2, 0, 0],
                [2, -2, 0, -1],
                [1, -1, 0, -1],
                [3, 2, 0, 2],
            ]
        ),
        "expected": 3,
    },
    {
        "name": "Example 4",
        "points": "\n".join(
            ",".join(map(str, l))
            for l in [
                [1, -1, -1, -2],
                [-2, -2, 0, 1],
                [0, 2, 1, 3],
                [-2, 3, -2, 1],
                [0, 2, 3, -2],
                [-1, -1, 1, -2],
                [0, -2, -1, 0],
                [-2, 2, 3, -1],
                [1, 2, 2, 0],
                [-1, -2, 0, -2],
            ]
        ),
        "expected": 8,
    },
]


class UnionFind:
    """
    https://en.wikipedia.org/wiki/Disjoint-set_data_structure

    https://github.com/shellfly/algs4-py/blob/master/algs4/uf.py
    This repository contains the Python source code for the algorithms in the
    textbook Algorithms, 4th Edition by Robert Sedgewick and Kevin Wayne.
    """

    def __init__(self, n):
        self.count = n
        self.id = list(range(n))
        self.sz = [1] * n

    def connected(self, p, q):
        return self.find(p) == self.find(q)

    def find(self, p):
        while self.id[p] != p:
            self.id[p] = self.id[self.id[p]]  # path compression
            p = self.id[p]
        return p

    def union(self, p, q):
        pId = self.find(p)
        qId = self.find(q)
        if pId == qId:
            return
        if self.sz[pId] < self.sz[qId]:
            self.id[pId] = qId
            self.sz[qId] += self.sz[pId]
        else:
            self.id[qId] = pId
            self.sz[pId] += self.sz[qId]
        self.count -= 1


def parse(s: str) -> list[list[int]]:
    return [tuple(int(i) for i in t.split(",")) for t in s.strip().splitlines()]


def distance(p1: Sequence[int], p2: Sequence[int]) -> int:
    return (
        abs(p1[0] - p2[0])
        + abs(p1[1] - p2[1])
        + abs(p1[2] - p2[2])
        + abs(p1[3] - p2[3])
    )


def neighbors(points, indici, p):
    return (q for q in indici[p + 1 :] if distance(points[p], points[q]) <= 3)


def constellations(points: str) -> int:
    points = parse(points)
    indici = list(range(len(points)))

    uf = UnionFind(len(points))

    for p in indici:
        for q in neighbors(points, indici, p):
            if not uf.connected(p, q):
                uf.union(p, q)

    return uf.count


run_tests_params(constellations, tests)


Test Example 1 passed, for constellations.
Test Example 2 passed, for constellations.
Test Example 3 passed, for constellations.
Test Example 4 passed, for constellations.
Success


In [13]:
with open("../input/day25.txt") as f:
    puzzle = f.read()

print(f"Part I: {constellations(puzzle)  }")

Part I: 327


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>327</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>A small glowing portal opens above the mug you prepared and just enough hot chocolate streams in to fill it. You suspect the reindeer has never encountered hot chocolate before, but seems to enjoy it anyway. You hope it works.</p>
<p>It's time to start worrying about that <em>integer underflow in time itself</em> you <a href="21">set up a few days ago</a>. You check the status of the device: "Insufficient chronal energy for activation. Energy required: <em class="star">50 stars</em>."</p>
<p>The reindeer bumps the device with its nose.</p>
<p>"Energy required: <em class="star">49 stars</em>."</p>
</article>

</main>


<link href="style.css" rel="stylesheet"></link>
<main>
<article>
<p>You use all <span class="day-success">fifty stars</span> to activate the underflow. As you begin to fall, the little reindeer looks up at you; its nose begins to glow red.</p><p>You go back in time so far that you wrap around and end up back in your own time again! Oddly, history books contain some new details that you don't recognize...</p>
<p class="aside">Congratulations!  You've finished every puzzle in Advent of Code 2018!  I hope you had as much fun solving them as I had making them for you.  I'd love to hear about your adventure; you can get in touch with me via contact info on <a href="http://was.tl/" target="_blank">my website</a> or through <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">If you'd like to see more things like this in the future, please consider <a href="/2018/support" target="_blank">supporting</a> Advent of Code and sharing it with others.</p>
<p class="aside">To hear about future projects, you can follow me on <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">I've <span style="border-bottom:1px dotted #ffff66;" title="Yep, just like that.  There's at least one in the description for each day.">highlighted</span> the easter eggs in each puzzle, just in case you missed any.  Hover your mouse over them, and the easter egg will appear.</p>
<p>You can <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+all+25+days+of+Advent+of+Code+2018%21&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+all+25+days+of+Advent+of+Code+2018%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this moment with your friends, or <a href="/2018">[Go Check on Your Calendar]</a>.
</p></article>
<style>
.calendar-bkg {
  background: linear-gradient(to bottom, rgba(255,255,255,0) 0%,rgba(255,255,255,.3) 100%);
}
.sf {
  position: fixed;
  animation: anim-sf0 linear 20s infinite;
  z-index: -1;
}
.sf:before {
  content: "*";
}
.sf.sf0 { color: #ffffff; }
.sf.sf1 { color: #cccccc; animation-name: anim-sf1; }
.sf.sf2 { color: #999999; animation-name: anim-sf2; }
.sf.sf3 { color: #666666; animation-name: anim-sf3; }
@keyframes anim-sf0 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(8em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(8em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf1 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(6em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(6em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf2 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(4em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(4em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf3 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(2em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(2em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
</style>
<div class="sf sf3" style="top:-4.96em; left:38.71vw; animation-delay:-9.84s;"></div><div class="sf sf1" style="top:-2.19em; left:2.98vw; animation-delay:-3.61s;"></div><div class="sf sf1" style="top:-2.82em; left:47.79vw; animation-delay:-17.52s;"></div><div class="sf sf1" style="top:-2.29em; left:65.79vw; animation-delay:-18.01s;"></div><div class="sf sf2" style="top:-3.79em; left:41.38vw; animation-delay:-18.55s;"></div><div class="sf sf1" style="top:-2.86em; left:73.95vw; animation-delay:-13.22s;"></div><div class="sf sf0" style="top:-1.38em; left:54.18vw; animation-delay:-15.88s;"></div><div class="sf sf0" style="top:-1.55em; left:33.80vw; animation-delay:-3.90s;"></div><div class="sf sf3" style="top:-4.63em; left:88.07vw; animation-delay:-0.55s;"></div><div class="sf sf0" style="top:-1.84em; left:80.79vw; animation-delay:-8.28s;"></div><div class="sf sf3" style="top:-4.83em; left:57.34vw; animation-delay:-7.92s;"></div><div class="sf sf0" style="top:-1.69em; left:33.43vw; animation-delay:-7.79s;"></div><div class="sf sf2" style="top:-3.65em; left:69.14vw; animation-delay:-10.63s;"></div><div class="sf sf0" style="top:-1.88em; left:44.74vw; animation-delay:-4.61s;"></div><div class="sf sf1" style="top:-2.94em; left:33.35vw; animation-delay:-15.18s;"></div><div class="sf sf3" style="top:-4.33em; left:17.19vw; animation-delay:-14.36s;"></div><div class="sf sf0" style="top:-1.89em; left:82.34vw; animation-delay:-18.65s;"></div><div class="sf sf0" style="top:-1.54em; left:59.93vw; animation-delay:-12.05s;"></div><div class="sf sf3" style="top:-4.68em; left:69.17vw; animation-delay:-0.27s;"></div><div class="sf sf2" style="top:-3.79em; left:95.33vw; animation-delay:-9.22s;"></div><div class="sf sf1" style="top:-2.73em; left:44.73vw; animation-delay:-8.35s;"></div><div class="sf sf3" style="top:-4.88em; left:10.66vw; animation-delay:-17.62s;"></div><div class="sf sf1" style="top:-2.31em; left:28.94vw; animation-delay:-17.25s;"></div><div class="sf sf2" style="top:-3.19em; left:54.81vw; animation-delay:-10.57s;"></div><div class="sf sf2" style="top:-3.44em; left:78.39vw; animation-delay:-11.56s;"></div><div class="sf sf3" style="top:-4.13em; left:9.81vw; animation-delay:-6.91s;"></div><div class="sf sf1" style="top:-2.23em; left:68.57vw; animation-delay:-4.79s;"></div><div class="sf sf3" style="top:-4.34em; left:56.80vw; animation-delay:-14.61s;"></div><div class="sf sf3" style="top:-4.00em; left:67.38vw; animation-delay:-18.63s;"></div><div class="sf sf2" style="top:-3.77em; left:56.82vw; animation-delay:-0.91s;"></div><div class="sf sf3" style="top:-4.30em; left:27.69vw; animation-delay:-15.42s;"></div><div class="sf sf2" style="top:-3.29em; left:73.01vw; animation-delay:-4.39s;"></div><div class="sf sf1" style="top:-2.35em; left:36.73vw; animation-delay:-11.18s;"></div><div class="sf sf0" style="top:-1.41em; left:1.97vw; animation-delay:-10.53s;"></div><div class="sf sf0" style="top:-1.41em; left:26.89vw; animation-delay:-19.56s;"></div><div class="sf sf0" style="top:-1.84em; left:14.45vw; animation-delay:-16.83s;"></div><div class="sf sf3" style="top:-4.59em; left:83.34vw; animation-delay:-8.18s;"></div><div class="sf sf0" style="top:-1.51em; left:72.77vw; animation-delay:-16.17s;"></div><div class="sf sf0" style="top:-1.53em; left:61.90vw; animation-delay:-7.65s;"></div><div class="sf sf3" style="top:-4.44em; left:60.30vw; animation-delay:-7.85s;"></div><div class="sf sf0" style="top:-1.01em; left:37.17vw; animation-delay:-1.38s;"></div><div class="sf sf3" style="top:-4.88em; left:56.92vw; animation-delay:-4.50s;"></div><div class="sf sf3" style="top:-4.66em; left:19.92vw; animation-delay:-19.11s;"></div><div class="sf sf0" style="top:-1.15em; left:53.13vw; animation-delay:-0.15s;"></div><div class="sf sf0" style="top:-1.82em; left:5.16vw; animation-delay:-1.13s;"></div><div class="sf sf3" style="top:-4.33em; left:9.83vw; animation-delay:-19.74s;"></div><div class="sf sf1" style="top:-2.20em; left:47.50vw; animation-delay:-11.48s;"></div><div class="sf sf2" style="top:-3.66em; left:80.61vw; animation-delay:-3.58s;"></div><div class="sf sf0" style="top:-1.95em; left:28.60vw; animation-delay:-18.45s;"></div><div class="sf sf3" style="top:-4.13em; left:58.87vw; animation-delay:-11.77s;"></div><div class="sf sf1" style="top:-2.82em; left:82.50vw; animation-delay:-8.33s;"></div><div class="sf sf0" style="top:-1.53em; left:17.41vw; animation-delay:-2.79s;"></div><div class="sf sf3" style="top:-4.42em; left:53.98vw; animation-delay:-14.68s;"></div><div class="sf sf0" style="top:-1.94em; left:34.91vw; animation-delay:-8.29s;"></div><div class="sf sf0" style="top:-1.75em; left:47.05vw; animation-delay:-9.44s;"></div><div class="sf sf2" style="top:-3.19em; left:4.93vw; animation-delay:-11.89s;"></div><div class="sf sf1" style="top:-2.01em; left:15.61vw; animation-delay:-14.35s;"></div><div class="sf sf2" style="top:-3.48em; left:65.55vw; animation-delay:-3.21s;"></div><div class="sf sf2" style="top:-3.10em; left:36.72vw; animation-delay:-11.80s;"></div><div class="sf sf3" style="top:-4.33em; left:70.17vw; animation-delay:-1.02s;"></div><div class="sf sf1" style="top:-2.32em; left:42.36vw; animation-delay:-2.59s;"></div><div class="sf sf0" style="top:-1.13em; left:58.33vw; animation-delay:-18.75s;"></div><div class="sf sf3" style="top:-4.68em; left:55.07vw; animation-delay:-14.83s;"></div><div class="sf sf1" style="top:-2.62em; left:53.73vw; animation-delay:-4.45s;"></div><div class="sf sf2" style="top:-3.39em; left:64.43vw; animation-delay:-12.93s;"></div><div class="sf sf1" style="top:-2.17em; left:15.80vw; animation-delay:-3.26s;"></div><div class="sf sf3" style="top:-4.40em; left:6.37vw; animation-delay:-8.54s;"></div><div class="sf sf0" style="top:-1.11em; left:90.81vw; animation-delay:-5.01s;"></div><div class="sf sf2" style="top:-3.54em; left:20.09vw; animation-delay:-16.54s;"></div><div class="sf sf3" style="top:-4.31em; left:13.06vw; animation-delay:-12.26s;"></div><div class="sf sf1" style="top:-2.14em; left:18.34vw; animation-delay:-7.90s;"></div><div class="sf sf0" style="top:-1.55em; left:12.76vw; animation-delay:-14.71s;"></div><div class="sf sf1" style="top:-2.04em; left:80.55vw; animation-delay:-9.00s;"></div><div class="sf sf0" style="top:-1.85em; left:22.39vw; animation-delay:-4.78s;"></div><div class="sf sf2" style="top:-3.76em; left:49.92vw; animation-delay:-2.13s;"></div><div class="sf sf0" style="top:-1.53em; left:50.47vw; animation-delay:-16.20s;"></div><div class="sf sf3" style="top:-4.74em; left:50.59vw; animation-delay:-15.45s;"></div><div class="sf sf3" style="top:-4.49em; left:77.91vw; animation-delay:-12.38s;"></div><div class="sf sf0" style="top:-1.71em; left:43.57vw; animation-delay:-12.38s;"></div><div class="sf sf1" style="top:-2.29em; left:19.68vw; animation-delay:-14.86s;"></div><div class="sf sf3" style="top:-4.92em; left:70.67vw; animation-delay:-0.32s;"></div><div class="sf sf0" style="top:-1.68em; left:62.48vw; animation-delay:-3.06s;"></div><div class="sf sf3" style="top:-4.19em; left:4.70vw; animation-delay:-16.19s;"></div><div class="sf sf3" style="top:-4.46em; left:81.97vw; animation-delay:-16.40s;"></div><div class="sf sf0" style="top:-1.83em; left:20.33vw; animation-delay:-12.58s;"></div><div class="sf sf2" style="top:-3.12em; left:37.23vw; animation-delay:-7.34s;"></div><div class="sf sf1" style="top:-2.02em; left:79.98vw; animation-delay:-2.46s;"></div><div class="sf sf2" style="top:-3.12em; left:24.05vw; animation-delay:-9.49s;"></div><div class="sf sf2" style="top:-3.67em; left:81.48vw; animation-delay:-2.43s;"></div><div class="sf sf3" style="top:-4.59em; left:63.40vw; animation-delay:-18.85s;"></div><div class="sf sf0" style="top:-1.56em; left:64.82vw; animation-delay:-5.26s;"></div><div class="sf sf3" style="top:-4.99em; left:6.66vw; animation-delay:-18.50s;"></div><div class="sf sf0" style="top:-1.22em; left:3.70vw; animation-delay:-10.66s;"></div><div class="sf sf2" style="top:-3.45em; left:9.55vw; animation-delay:-6.50s;"></div><div class="sf sf2" style="top:-3.46em; left:74.27vw; animation-delay:-10.12s;"></div><div class="sf sf1" style="top:-2.61em; left:58.08vw; animation-delay:-15.83s;"></div><div class="sf sf1" style="top:-2.06em; left:52.60vw; animation-delay:-16.43s;"></div><div class="sf sf1" style="top:-2.29em; left:51.07vw; animation-delay:-0.72s;"></div><div class="sf sf0" style="top:-1.92em; left:50.07vw; animation-delay:-11.17s;"></div><div class="sf sf1" style="top:-2.72em; left:76.06vw; animation-delay:-8.82s;"></div>
</main>
